In [11]:
import sys
from collections import defaultdict
import xlrd
import gurobipy as grb
from gurobipy import GRB

import numpy as np
import math

np.random.seed(248745)

In [12]:
class Hotspot():
    def __init__(self, number, profits, area, uuv_obs_rate):
        self.number = number
        self.profits = profits
        self.area = area
        self.lam = uuv_obs_rate/area

In [13]:
distance = np.random.randint(0, 1000, size=(7,7))

##build all the hotspot data
uuv_obs_rate = (5**2)*math.pi #arbitrarily selected observation rate
all_hotspots = [Hotspot(0, 0, 0.1, uuv_obs_rate), ##This is the init position
                Hotspot(1, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(2, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(3, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(4, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(5, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                Hotspot(6, np.random.randint(0,100), np.random.randint(500,2500), uuv_obs_rate),
                ]

In [33]:
hotspot_idx = [i.number for i in all_hotspots]
points = [(i,j) for i in hotspot_idx for j in hotspot_idx]
profits = [i.profits for i in all_hotspots]
lam = [i.lam for i in all_hotspots]
# Distance = 50000
Distance = 3000
n = len(all_hotspots)

In [32]:
def profit(idx, h_time):
    return 1 - math.exp(-lam[idx] * h_time)

In [60]:
opt_model = grb.Model(name="MILP Model")

# <= Variables
##x_vars = x_ij travel from hotspot i to j
x_vars = {}
for i in range(n):
   for j in range(n):
     x_vars[i,j] = opt_model.addVar(vtype=grb.GRB.BINARY,
                          name='e'+str(i)+'_'+str(j))

##u = boolean if visit hotspot i
u={}
for i in range(1,n):
    u[i]=opt_model.addVar(vtype=grb.GRB.INTEGER,
                          name='e'+str(i))
    
##Introduce hotspot_time variable
##Create profits function
npts = 300
ptf = np.empty((n, npts))
ptu = np.arange(0, npts, 1).tolist()

for i in range(0,n):
    for p in range(npts):
        ptf[i,p] = profit(i, ptu[p])

t={}
for i in range(1,n):
    t[i]=opt_model.addVar(lb = 0.0, ub = 300, name="ti")
    opt_model.setPWLObj(t[i], ptu, ptf[i,:].tolist())


# <= Constraint (Mandatory Edges and excluding vertexes) Eq(1)
opt_model.addConstr(grb.quicksum([x_vars[1,j] for j in range(1,n)])  == 1)
opt_model.addConstr(grb.quicksum([x_vars[i,n-1] for i in range(n-1)])  == 1)
# opt_model.addConstr((grb.quicksum(x_vars[i,i] for i in range(n-1)))  == 0)


# <= Constraint (Distance) Eq(3)
# for i in range(n-1):
#   opt_model.addConstr(grb.quicksum([x_vars[i,j]*distance(points, i, j) for j in range(1,n)]) <= Distance)
# opt_model.addConstr(grb.quicksum([x_vars[i,j]*distance(points, i,j) for j in range(1,n) for i in range(n-1)]) <= Distance)
opt_model.addConstr(grb.quicksum([x_vars[i,j]*t[j]
                                + x_vars[i,j]*distance[i,j] for j in range(1,n) for i in range(n-1)]) <= Distance)


# <= Constraint (Equality & Single edge in and out) Eq(2)
for k in range(1, n-1):
#   opt_model.addConstr(grb.quicksum(x_vars[i,k] for i in range(n-1))
#                       == grb.quicksum(x_vars[k,j] for j in range(1, n)) <=1)

    one = grb.quicksum([x_vars[i,k] for i in range(n-1)])
    two = grb.quicksum([x_vars[k,j] for j in range(1, n)])
    opt_model.addConstr(one == two)
    opt_model.addConstr(one <= 1)

# <= Constraint (Subtour elimination) Eq(4) Eq(5)
for i in range(1,n):
    opt_model.addConstr(2 <= u[i])
    opt_model.addConstr(u[i] <= n)
#   opt_model.addConstr(2 <= u[i] <= n)

for i in range(1,n):
    for j in range(1,n):
        opt_model.addConstr((u[i] - u[j] +1 <= (n-1)*(1-x_vars[j,i])))

        
# <= objective (maximize) Eq(1)
objective = grb.quicksum([x_vars[i,j]
                         for i in range(1, n-1)
                         for j in range(1, n)])

opt_model.ModelSense = grb.GRB.MAXIMIZE
opt_model.setObjective(objective)
opt_model.optimize()

solution = opt_model.getAttr('x', x_vars )
print (solution)

select = grb.tuplelist((i,j) for i,j in x_vars.keys() if x_vars[i,j].X > 0.5)
print (select)

total = 0
for z in range(len(select)):
    total += distance[select[z][0], select[z][1]] + profits[select[z][0]]
    
print ("TOTAL ANSWER IS : ", total)

##now do the second optimization trick   
# for i in range(0,n):
#     for p in range(npts):
#         ptf[i,p] = -ptf[i,p]
        
# t={}
# for i in range(1,n):
#     t[i]=opt_model.addVar(lb = 0.0, ub = 300, name="ti")
#     opt_model.setPWLObj(t[i], ptu, ptf[i,:].tolist())
    
# opt_model.optimize()


# # opt_model.update()
# # opt_model.computeIIS()
# # opt_model.write("model.ilp")
# solution = opt_model.getAttr('x', x_vars )

# print (solution)

# select = grb.tuplelist((i,j) for i,j in x_vars.keys() if x_vars[i,j].X > 0.5)
# print (select)

# total = 0
# for z in range(len(select)):
#     total += distance[select[z][0], select[z][1]] + profits[select[z][0]]
    
# print ("TOTAL ANSWER IS : ", total)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 60 rows, 61 columns and 200 nonzeros
Model fingerprint: 0x2fe47786
Model has 1 quadratic constraint
Variable types: 6 continuous, 55 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [8e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+02]
  RHS range        [1e+00, 7e+00]
  QRHS range       [3e+03, 3e+03]
Presolve removed 24 rows and 19 columns
Presolve time: 0.00s
Presolved: 68 rows, 73 columns, 369 nonzeros
Variable types: 0 continuous, 73 integer (31 binary)
Found heuristic solution: objective 2.0000000

Root relaxation: objective -5.000000e+00, 25 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.00000    0    4    2.00000    5.00000   150%     -    0s
H  

In [55]:
select = grb.tuplelist((i,j) for i,j in x_vars.keys() if x_vars[i,j].X > 0.5)
print (select)

<gurobi.tuplelist (6 tuples, 2 values each):
 ( 0 , 1 )
 ( 1 , 2 )
 ( 2 , 3 )
 ( 3 , 4 )
 ( 4 , 5 )
 ( 5 , 6 )
>


In [56]:
total = 0
for z in range(len(select)):
    total += distance[select[z][0], select[z][1]] + profits[select[z][0]]
    
print (total)

2702


In [53]:
t

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>}